In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Charger les données à partir du fichier Excel
chemin_fichier = "Truss.xlsx"
data = pd.read_excel(chemin_fichier)

In [3]:
# Collecte des données d'entraînement
X= data[['csa1','csa2','csa3','csa4','csa5','csa6','csa7','csa8','csa9','csa10',]]  # Vecteur des aires des sections transversales (100 exemples)
y = data[['stress1','stress2','stress3','stress4', 'stress5', 'stress6', 'stress7', 'stress8', 'stress9', 'stress10']]  # Vecteur des contraintes correspondantes

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print("Training set size: ", len(X_train))
print("Validation set size: ", len(X_val))
print("Test set size: ", len(X_test))

Training set size:  12800
Validation set size:  3200
Test set size:  4000


In [5]:
# Création du modèle du réseau de neurones
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(10,), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

In [6]:
# Compilation du modèle
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                704       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 5,514
Trainable params: 5,514
Non-trainable params: 0
_________________________________________________________________


In [7]:

# Callback pour l'early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [8]:

# Entraînement du modèle
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])


Epoch 1/100
400/400 [==============================] - 1s 3ms/step - loss: 25423388893773824.0000 - accuracy: 0.1631 - val_loss: 23011028170178560.0000 - val_accuracy: 0.0872
Epoch 2/100
400/400 [==============================] - 1s 2ms/step - loss: 25422935774724096.0000 - accuracy: 0.1077 - val_loss: 23010087572340736.0000 - val_accuracy: 0.0872
Epoch 3/100
400/400 [==============================] - 1s 2ms/step - loss: 25421318719537152.0000 - accuracy: 0.0846 - val_loss: 23007671653236736.0000 - val_accuracy: 0.0872
Epoch 4/100
400/400 [==============================] - 1s 2ms/step - loss: 25418041659490304.0000 - accuracy: 0.0840 - val_loss: 23003378833424384.0000 - val_accuracy: 0.0872
Epoch 5/100
400/400 [==============================] - 1s 2ms/step - loss: 25412750259781632.0000 - accuracy: 0.0833 - val_loss: 22996940677447680.0000 - val_accuracy: 0.0872
Epoch 6/100
400/400 [==============================] - 1s 2ms/step - loss: 25405210444693504.0000 - accuracy: 0.0834 - val_lo

In [9]:
# Évaluation du modèle sur l'ensemble de test
loss, accuracy = model.evaluate(X_test, y_test)
print("Perte :", loss)
print("Précision :", accuracy)

125/125 [==============================] - 0s 1ms/step - loss: 15583042219278336.0000 - accuracy: 0.5288
Perte : 1.5583042219278336e+16
Précision : 0.5287500023841858


In [10]:
# Exemple de prédiction sur un vecteur d'entrée donné
#X_example = np.random.rand(1, 10)  # Vecteur d'entrée de test
prediction = model.predict(X_test)
print("Prédiction :", prediction)


Prédiction : [[ 1.25585552e+08  9.61540640e+07  3.52742840e+07 ...  3.25413840e+07
  -1.29462376e+08 -5.22624920e+07]
 [ 1.30821088e+08  1.00162592e+08  3.67448320e+07 ...  3.38979960e+07
  -1.34859520e+08 -5.44412680e+07]
 [ 1.26573576e+08  9.69105280e+07  3.55517960e+07 ...  3.27973860e+07
  -1.30480872e+08 -5.26736560e+07]
 ...
 [ 1.27908136e+08  9.79323040e+07  3.59266400e+07 ...  3.31432020e+07
  -1.31856664e+08 -5.32290400e+07]
 [ 1.28254224e+08  9.81972960e+07  3.60238560e+07 ...  3.32328760e+07
  -1.32213424e+08 -5.33730640e+07]
 [ 1.29967456e+08  9.95090400e+07  3.65050640e+07 ...  3.36768080e+07
  -1.33979552e+08 -5.40860280e+07]]


In [11]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score
mean_absolute_error(y_test,prediction)

52034473.8514625